In [327]:
import pandas as pd
import numpy as np

In [328]:
eco = pd.read_csv(r'/kaggle/input/flight-price-prediction/economy.csv')

bus = pd.read_csv(r'/kaggle/input/flight-price-prediction/business.csv')

In [329]:
# eco.head()

# bus.head()

In [330]:
# eco.info()

# bus.info()

In [331]:
df = pd.concat([eco, bus], ignore_index=True)

# df.info()

In [332]:
df['Flight-Id'] = df['ch_code'] + '-' + df['num_code'].astype(str)
df = df.drop(['ch_code', 'num_code'], axis = 1)


df['Class'] = np.where(df.index.isin(eco.index), 'Economy', 'Business')


df['Source'] = df['from']
df = df.drop('from', axis = 1)


df['Destination'] = df['to']
df = df.drop('to', axis = 1)

In [333]:
import re

def convert_time(time_str):
    pattern = r'(\d+)h\s*(\d+)m'
    match = re.match(pattern, time_str)
    if match:
        hours = int(match.group(1))
        minutes = int(match.group(2))
        decimal_hours = hours + (minutes / 60)
        return round(decimal_hours, 2)

df['Time'] = df['time_taken'].apply(convert_time)
df = df.drop('time_taken', axis = 1)

In [334]:
df['Time'] = df['Time'].fillna(df['Time'].mean())

In [335]:
for index, value in df['dep_time'].items():
    hour = int(value.split(':')[0])
    if 5 <= hour < 8 :
        df.at[index, 'dep_time'] = 'Early-Morning'
    elif 8 <= hour < 12 :
        df.at[index, 'dep_time'] = 'Morning'
    elif 12 <= hour < 16 :
        df.at[index, 'dep_time'] = 'Afternoon'
    elif 16 <= hour < 19 :
        df.at[index, 'dep_time'] = 'Evening'
    elif 19 <= hour < 23 :
        df.at[index, 'dep_time'] = 'Night'
    else :
        df.at[index, 'dep_time'] = 'Mid-Night'
        
for index, value in df['arr_time'].items():
    hour = int(value.split(':')[0])
    if 5 <= hour < 8 :
        df.at[index, 'arr_time'] = 'Early-Morning'
    elif 8 <= hour < 12 :
        df.at[index, 'arr_time'] = 'Morning'
    elif 12 <= hour < 16 :
        df.at[index, 'arr_time'] = 'Afternoon'
    elif 16 <= hour < 19 :
        df.at[index, 'arr_time'] = 'Evening'
    elif 19 <= hour < 23 :
        df.at[index, 'arr_time'] = 'Night'
    else :
        df.at[index, 'arr_time'] = 'Mid-Night'

In [336]:
def convert_stops(value):
    st = value.split('-')[0]
    if st == 'non' :
        return 0
    else :
        return st

df['Stops'] = df['stop'].apply(convert_stops)
df = df.drop('stop', axis = 1)

In [337]:
new_column_order = ['airline', 'Flight-Id', 'Class', 'date', 'Source', 'Destination', 'Stops' ,'dep_time', 'arr_time', 'Time', 'price']
df = df[new_column_order]

In [338]:
# df.head()

# df.info()

In [339]:
df.to_csv('Clean_Dataset.csv', index = False)